# Mask R-CNN - Inspect FNV Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
TRAIN_DIR = os.path.abspath("../train")
# Import Mask RCNN
sys.path.append(TRAIN_DIR)  # To find local version of the library

import visualize


#import general utils functions
import utils.general_utils as gu
#import project related utils
import utils.project_utils as pu


%matplotlib inline 


#Import from inference package
from fnv_inference import model as modellib
from fnv_inference.fnv import FNVDataset, FNVConfig

In [ ]:
train_config_path = os.path.join(TRAIN_DIR, 'train_config.ini')

In [ ]:
import logging
logger = logging.getLogger('fnv_performance_analysis')
LOGI = logger.info
LOGD = logger.debug
LOGE = logger.error
import logging.config
log_conf = pu.get_logconf_path(train_config_path)
logging.config.fileConfig(log_conf)

In [ ]:
train_config = pu.get_configParser(train_config_path)

In [ ]:
# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
project_folder = train_config['local_variables']['project_folder']
model_path = os.path.join(project_folder, train_config['project_folders']['deployment'], train_config['model_files']['fnv_model'])

## Configurations

In [ ]:
config = FNVConfig()

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
# DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Test Dataset

In [ ]:
meta_dir = os.path.join(project_folder, train_config['project_folders']['meta_data'])
dataset_dir = os.path.join(project_folder, train_config['project_folders']['data'])
split_data_filepath = os.path.join(meta_dir, train_config['files']['split_data_file'])
MODEL_DIR = os.path.join(project_folder, train_config['project_folders']['deployment'])
weights_path = os.path.join(MODEL_DIR, train_config['model_files']['fnv_model'])

In [ ]:
sku_product_ids = pu.get_sku_ids(train_config_path)

In [ ]:
split_data = gu.load_json(split_data_filepath)

In [ ]:
# Load test dataset
dataset = FNVDataset()
mode = "test"
dataset.load_csku(dataset_dir, meta_dir, sku_product_ids, split_data, mode)

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_balloon.h5"

# Or, load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Run Detection

In [ ]:
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
print("gt_class_id", gt_class_id)
# print("gt_bbox", gt_bbox)
# print("gt_mask", gt_mask)

In [ ]:
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                           dataset.image_reference(image_id)))

    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax,
                                title="Predictions")
    print("gt_class_id", gt_class_id)